In [1]:
using Tricks

In [2]:
?static_hasmethod

search: static_hasmethod



```
static_hasmethod(f, type_tuple::Type{<:Tuple)
```

Like `hasmethod` but runs at compile-time (and does not accept a worldage argument).


In [3]:
struct A end
struct B end
abstract type FooBar end
struct Foo <: FooBar x::Int end
foomethod(foo::Foo) = nothing
struct Bar <: FooBar x::Int end

In [4]:
foo = Foo(23)
hasmethod(foomethod, Tuple{Foo})

true

In [5]:
attr_dynamic(fb::T) where T <: FooBar = hasmethod(foomethod, Tuple{T}) ? A() : B()
f_dynamic(fb::FooBar) = f(attr_dynamic(fb), fb)
f(::A, fb) = fb.x + 100
f(::B, fb) = fb.x - 100

f (generic function with 2 methods)

In [6]:
f_dynamic(Foo(23))

123

In [7]:
f_dynamic(Bar(-23))

-123

In [8]:
@code_typed f_dynamic(Foo(23))

CodeInfo(
1 ─       invoke Base.to_tuple_type(Tuple{Foo}::Any)::Type{Tuple{Foo}}
│   %2  = $(Expr(:foreigncall, :(:jl_gf_invoke_lookup), Any, svec(Any, UInt64), 0, :(:ccall), Tuple{typeof(foomethod), Foo}, 0xffffffffffffffff, 0xffffffffffffffff))::Any
│   %3  = (%2 === Base.nothing)::Bool
│   %4  = Core.Intrinsics.not_int(%3)::Bool
└──       goto #3 if not %4
2 ─       goto #4
3 ─       goto #4
4 ┄ %8  = φ (#2 => $(QuoteNode(A())), #3 => $(QuoteNode(B())))::Union{A, B}
│   %9  = (isa)(%8, A)::Bool
└──       goto #6 if not %9
5 ─ %11 = Base.getfield(fb, :x)::Int64
│   %12 = Base.add_int(%11, 100)::Int64
└──       goto #9
6 ─ %14 = (isa)(%8, B)::Bool
└──       goto #8 if not %14
7 ─ %16 = Base.getfield(fb, :x)::Int64
│   %17 = Base.sub_int(%16, 100)::Int64
└──       goto #9
8 ─       Core.throw(ErrorException("fatal error in type inference (type bound)"))::Union{}
└──       unreachable
9 ┄ %21 = φ (#5 => %12, #7 => %17)::Int64
└──       return %21
) => Int64

In [9]:
foo = Foo(23)
static_hasmethod(foomethod, Tuple{Foo})

true

In [10]:
attr_static(fb::T) where T <: FooBar = static_hasmethod(foomethod, Tuple{T}) ? A() : B()
f_static(fb::FooBar) = f(attr_static(fb), fb)

f_static (generic function with 1 method)

In [11]:
f_static(Foo(23))

123

In [12]:
f_static(Bar(-23))

-123

In [13]:
@code_typed f_static(Foo(23))

CodeInfo(
1 ─ %1 = Base.getfield(fb, :x)::Int64
│   %2 = Base.add_int(%1, 100)::Int64
└──      return %2
) => Int64

In [14]:
@code_typed f_static(Bar(-23))

CodeInfo(
1 ─ %1 = Base.getfield(fb, :x)::Int64
│   %2 = Base.sub_int(%1, 100)::Int64
└──      return %2
) => Int64

In [15]:
foomethod(fb::Bar) = nothing

foomethod (generic function with 2 methods)

In [16]:
f_static(Bar(-23))

77

In [17]:
@code_typed f_static(Bar(-23))

CodeInfo(
1 ─ %1 = Base.getfield(fb, :x)::Int64
│   %2 = Base.add_int(%1, 100)::Int64
└──      return %2
) => Int64